In [2]:
import pandas as pd

bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Bridges_processed.csv" # ROBOTA

bridges_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Bridges_processed.csv" # DOMA


df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')

In [3]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')


In [67]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
filtered_bridges_SSC = df_bridges_all[
    ((df_bridges_all['Správca_úseku'] == "SSC - IVSC ZA") |
     (df_bridges_all['Správca_úseku'] == "SSC - IVSC KE") |
     (df_bridges_all['Správca_úseku'] == "SSC - IVSC BA")) &
    (df_bridges_all['n_2023'] > 0) &
    (df_bridges_all['ID_mosta'] != "Nezistený")
].copy()
# filtered_bridges_SSC.to_excel('filtered_bridges_SSC.xlsx', index=False)
# Display the filtered DataFrame
filtered_bridges_SSC.shape

(1364, 51)

In [68]:
filtered_bridges_SSC.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Most_DC', 'Názov_mosta', 'ID_DC',
       'správcovské_číslo', 'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška',
       'Druh_konštrukcie', 'Material', 'Normálna_zaťažiteľnosť',
       'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
       'Spôsob_stanovenia', 'Predmet_premostenia', 'Rok_postavenia',
       'Zaťažovacia_trieda_STN', 'Počet_otvorov', 'Dĺžka_premostenia',
       'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023'],
      

In [69]:
import pandas as pd

# Assuming filtered_bridges_SSC is your DataFrame

# Select columns starting with "n_"
condition_columns = filtered_bridges_SSC.filter(like='n_').columns

# Concatenate values of selected columns into a new column "STS"
filtered_bridges_SSC['STS'] = filtered_bridges_SSC[condition_columns].apply(
    lambda x: ','.join(x.dropna().astype(str)), axis=1)

In [70]:
filtered_bridges_SSC["STS"]

0       3,3,3,3,3,5,5,5,5,5,5,5
1       3,3,3,3,3,6,6,6,6,6,6,6
2       4,4,4,4,4,4,4,4,4,4,4,4
3       3,3,3,3,3,3,3,3,3,3,3,4
4       3,3,4,4,4,4,4,4,4,5,5,5
                 ...           
8387    0,0,0,0,0,0,0,0,0,0,2,2
8388    0,0,0,0,0,0,0,0,0,0,6,6
8389    0,0,0,0,0,0,0,0,0,0,3,3
8480    0,0,0,0,0,0,0,0,0,0,0,1
8482    0,0,0,0,0,0,0,0,0,0,0,1
Name: STS, Length: 1364, dtype: object

In [71]:
# Split the "STS" column into lists of integers
filtered_bridges_SSC['STS_List'] = filtered_bridges_SSC['STS'].apply(
    lambda x: [int(n) for n in x.split(',')])

# Create a new column "zeros" containing the number of zeros in each STS_List
filtered_bridges_SSC['zeros'] = filtered_bridges_SSC['STS_List'].apply(
    lambda x: x.count(0))

In [72]:
filtered_bridges_SSC['zeros'].value_counts()

zeros
0     1250
4       29
2       17
8       17
9       14
7       13
5       12
1        4
10       4
11       3
3        1
Name: count, dtype: int64

In [73]:
def compute_slope(y):
    # Filter out zeros and get the indices of non-zero values
    y_filtered = np.array(y)[np.array(y) > 0]
    x_filtered = np.arange(len(y))[np.array(y) > 0]

    # Return NaN slope if not enough data
    if len(y_filtered) <= 1:
        return np.nan

    # Compute slope
    slope, intercept, r_value, p_value, std_err = linregress(
        x_filtered, y_filtered)

    # Calculate the scaling factor as the ratio of non-zero values to the total values
    scaling_factor = len(y_filtered) / len(y)
    scaled_slope = slope * scaling_factor

    return scaled_slope


# Applying the compute_slope function to each STS list and creating a scaled degradation rate column
filtered_bridges_SSC['Scaled_Degradation_Rate'] = filtered_bridges_SSC['STS_List'].apply(
    compute_slope)

# Optionally, to view or analyze the distribution of scaled degradation rates, you might add:
print(filtered_bridges_SSC['Scaled_Degradation_Rate'].describe())

count    1361.000000
mean        0.063711
std         0.136913
min        -0.531469
25%         0.000000
50%         0.069930
75%         0.125874
max         0.559441
Name: Scaled_Degradation_Rate, dtype: float64


In [79]:
import pandas as pd

# Assuming filtered_bridges_SSC is your DataFrame

# Select 100 random rows
random_sample = filtered_bridges_SSC.sample(n=100)

# Save the sampled DataFrame to an Excel file
random_sample.to_excel('random_sample.xlsx', index=False)

# Optionally, display the sampled DataFrame
print(random_sample)

     ID_mosta        Trieda_PK Číslo_PK          Most_DC  \
202      M761  cesta I. triedy       18  Dilatačný celok   
691      M416  cesta I. triedy       51  Dilatačný celok   
237     M4018  cesta I. triedy       18  Dilatačný celok   
1053    M3584  cesta I. triedy       64  Dilatačný celok   
282      M973  cesta I. triedy       18  Dilatačný celok   
...       ...              ...      ...              ...   
11      M2128  cesta I. triedy        2  Dilatačný celok   
1157    M6084  cesta I. triedy       18  Dilatačný celok   
940     M5386  cesta I. triedy      61B  Dilatačný celok   
148      M262  cesta I. triedy       10  Dilatačný celok   
1545    M6681  cesta I. triedy       74  Dilatačný celok   

                                            Názov_mosta     ID_DC  \
202   18_276 Portál tunela medzi Strečnom a Dubnou S...   M761.01   
691               51_142 Most cez jarok, k.ú. Horná Seč   M416.01   
237   18_321 Most cez potok Bystré, k.ú. Bystré - Če...  M4018.01   
105

In [75]:
# Sort the DataFrame by the maximum value of Degradation_Rate
sorted_df = filtered_bridges_SSC.sort_values(
    by='Scaled_Degradation_Rate', ascending=False)

In [76]:
filtered_bridges_SSC['Scaled_Degradation_Rate']

0       0.244755
1       0.367133
2       0.000000
3       0.038462
4       0.164336
          ...   
8387    0.000000
8388    0.000000
8389    0.000000
8480         NaN
8482         NaN
Name: Scaled_Degradation_Rate, Length: 1364, dtype: float64

In [77]:
sorted_df[["ID_mosta", "STS", 'Scaled_Degradation_Rate']].head(10)

,ID_mosta,STS,Scaled_Degradation_Rate
1690,M6287,"1,1,1,1,1,1,1,5,5,5,6,6",0.559441
23,M5509,"2,2,2,2,2,6,6,6,6,6,6,6",0.489510
25,M6110,"2,2,2,2,2,6,6,6,6,6,6,6",0.489510
1370,M4413,"2,2,2,2,2,2,2,5,5,5,6,7",0.475524
47,M2061,"2,2,2,2,2,4,4,4,6,6,6,6",0.468531
293,M4815,"2,2,2,4,4,4,4,4,6,6,6,7",0.451049
115,M3602,"2,2,2,2,7,6,6,6,6,6,6,6",0.437063
1687,M904,"2,2,2,2,2,2,2,5,5,5,6,6",0.437063
1358,M5096,"3,3,3,3,3,3,3,5,6,6,7,7",0.426573
114,M3740,"3,3,4,4,5,5,5,5,7,7,7,7",0.405594


In [78]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

# Define a function to compute the slope


def compute_slope(y):
    # Filter out zeros and get the indices of non-zero values
    y_filtered = np.array(y)[np.array(y) > 0]
    x_filtered = np.arange(len(y))[np.array(y) > 0]

    print("Filtered Y values:", y_filtered)
    print("Filtered X indices:", x_filtered)

    # Return NaN slope if not enough data
    if len(y_filtered) <= 1:
        print("Insufficient data points for computing slope.")
        return np.nan

    # Compute slope using linear regression
    slope, intercept, r_value, p_value, std_err = linregress(
        x_filtered, y_filtered)
    print("Linear Regression:")
    print("Slope:", slope)
    print("Intercept:", intercept)
    print("R-value:", r_value)
    print("P-value:", p_value)
    print("Standard Error:", std_err)
    return slope


# Define an example value for the 'STS' column
STS_example = '0,0,0,0,0,0,0,0,0,1,3,3'
# STS_example = "2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 6, 7"
STS_example = "1,1,1,1,1,1,1,5,5,5,6,6"

# Split the 'STS' string into a list of integers
STS_list = [int(n) for n in STS_example.split(',')]
print("STS List:", STS_list)

# Apply the compute_slope function to the list of integers
degradation_rate = compute_slope(STS_list)

# Print the result
print("Degradation Rate:", degradation_rate)

STS List: [1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 6, 6]
Filtered Y values: [1 1 1 1 1 1 1 5 5 5 6 6]
Filtered X indices: [ 0  1  2  3  4  5  6  7  8  9 10 11]
Linear Regression:
Slope: 0.5594405594405594
Intercept: -0.24358974358974317
R-value: 0.8809668042591369
P-value: 0.00015361232008200698
Standard Error: 0.09502087334698474
Degradation Rate: 0.5594405594405594
